In [1]:
import cobra
import pandas as pd

In [35]:
model = cobra.io.read_sbml_model('../assets/iJO1366.xml')
model_name = 'iJO1366'
media = pd.read_csv('../assets/media_sets_names.txt',sep='\t')

# get the base medium
medium = model.medium
medium.pop('EX_glc__D_e')
medium.pop('EX_nh4_e')
medium.pop('EX_o2_e')
medium_base = medium.copy()


In [43]:
obj_values = []
for idx,row in media.iterrows():
    m = medium_base.copy();
    m[row.carbon] = 10;
    m[row.nitrogen] = 1000;
    if row.electron != 'None':
        m[row.electron] = 1000;
    
    model.medium = m;
    f = model.slim_optimize()
    obj_values.append(f)
    
media['model'] = model_name;
media['growth_rate'] = obj_values;

KeyboardInterrupt: 

In [74]:
media

,carbon,nitrogen,electron
0,EX_12ppd__S_e,EX_23camp_e,EX_no2_e
1,EX_12ppd__S_e,EX_23camp_e,EX_no3_e
2,EX_12ppd__S_e,EX_23camp_e,EX_o2_e
3,EX_12ppd__S_e,EX_23camp_e,None
4,EX_12ppd__S_e,EX_23ccmp_e,EX_no2_e
...,...,...,...
66955,EX_xylu__L_e,EX_xmp_e,None
66956,EX_xylu__L_e,EX_xtsn_e,EX_no2_e
66957,EX_xylu__L_e,EX_xtsn_e,EX_no3_e
66958,EX_xylu__L_e,EX_xtsn_e,EX_o2_e


In [64]:
len(media.nitrogen.unique()) * len(media.carbon.unique()) * 7

117180

In [59]:
#[x for x in model.exchanges if x.id == 'EX_no3_e']

In [77]:
media_components = {}
csources = media.carbon.unique().tolist()
nsources = media.nitrogen.unique().tolist()

media_components['electron_acceptors'] = ['EX_tmao_e','EX_dmso_e','EX_fum_e','EX_o2_e','EX_no2_e','EX_no3_e','None']
media_components['nitrogen_sources'] = nsources;
media_components['carbon_sources'] = csources;


In [78]:
media_sets = {'carbon':[],'nitrogen':[],'electron':[]}
for csource in media_components['carbon_sources']:
    for nsource in media_components['nitrogen_sources']:
        for eacceptor in media_components['electron_acceptors']:
            media_sets['carbon'].append(csource)
            media_sets['nitrogen'].append(nsource)
            media_sets['electron'].append(eacceptor)

In [80]:
media_new = pd.DataFrame(media_sets)

In [81]:
media_new.to_csv('../assets/media.C180N93E7.txt',sep='\t')

In [86]:
media_new.sample(1000).to_csv('../tests/media.C180N93E7.sample1000.txt',sep='\t')